In [1]:
import pickle
pickle_in = open("xtrain.pickle","rb")
xtrain = pickle.load(pickle_in)
pickle_in.close()

In [2]:
import pickle
pickle_in = open("xtest.pickle","rb")
xtest = pickle.load(pickle_in)
pickle_in.close()

In [3]:
import pickle
pickle_in = open("ytrain.pickle","rb")
ytrain = pickle.load(pickle_in)
pickle_in.close()

In [4]:
import gensim
from gensim.models import word2vec,KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary = True)

/home/vidooly/ml/software/python_env/env_ten/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
emb_dim = word_vectors["the"].shape
print(emb_dim)

(300,)


In [20]:
import numpy as np
def sec2vec(x):
    maxlen = 10
    state_vec = np.zeros((len(x),maxlen,emb_dim[0]))
    for i in range(len(x)):
        
        for j in range(maxlen):
            try:
                state_vec[i][j] = word_vectors[x[i][j].lower()]
            except:
                state_vec[i][j] = np.zeros((300,))
    return state_vec

In [21]:
stae_vec = sec2vec(xtrain)

In [23]:
embedding_mat_test = sec2vec(xtest)

In [25]:
import keras
from keras.models import Sequential
from keras.layers import *

Using TensorFlow backend.


In [48]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,300),return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences = False))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.summary()

ValueError: ('`cell` should have a `call` method. The RNN was passed:', 64)

In [31]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("best_model_lstm.h5",monitor = "val_loss",verbose = True,save_best_only=True)
earlystop = EarlyStopping(monitor = "val_acc",patience = 14)

In [34]:
model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ["acc"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [35]:
model.fit(stae_vec,ytrain,epochs = 100,batch_size = 64,shuffle = True,validation_split = 0.2,callbacks = [checkpoint,earlystop])


Train on 32000 samples, validate on 8000 samples
Epoch 1/100
32000/32000 [==============================] - 7s 204us/step - loss: 0.6055 - acc: 0.6556 - val_loss: 0.5828 - val_acc: 0.6737

Epoch 00001: val_loss improved from inf to 0.58276, saving model to best_model.h5
Epoch 2/100
32000/32000 [==============================] - 6s 181us/step - loss: 0.5711 - acc: 0.6850 - val_loss: 0.5734 - val_acc: 0.6842

Epoch 00002: val_loss improved from 0.58276 to 0.57339, saving model to best_model.h5
Epoch 3/100
32000/32000 [==============================] - 5s 165us/step - loss: 0.5560 - acc: 0.7002 - val_loss: 0.5596 - val_acc: 0.6963

Epoch 00003: val_loss improved from 0.57339 to 0.55957, saving model to best_model.h5
Epoch 4/100
32000/32000 [==============================] - 5s 167us/step - loss: 0.5443 - acc: 0.7067 - val_loss: 0.5553 - val_acc: 0.7001

Epoch 00004: val_loss improved from 0.55957 to 0.55534, saving model to best_model.h5
Epoch 5/100
32000/32000 [=========================

In [38]:
model.load_weights("best_model.h5")
pred = model.predict(embedding_mat_test)

In [46]:
classes = []
for i in pred:
    if i>0.5:
        classes.append('pos')
    else:
        classes.append('neg')

In [47]:
import pandas as pd
pd.DataFrame(classes).to_csv("Predicted.csv",index = False)